# NER Based Skill Extraction

For skill extraction, we built a custom Skill-NER model using spaCy's PhraseMatcher and a curated dictionary of 450+ skills. This allowed us to define our own ‘SKILL’ entity type without requiring a large annotated dataset. In addition, spaCy’s statistical NER (ORG, GPE, DATE) was used to extract metadata from resumes and job postings. This combination of rule-based and statistical NER forms a hybrid NER approach aligned with Module 4 of the course.

### Read cleaned resume file

In [1]:
import pandas as pd
import numpy as np
text= pd.read_csv("cleaned_resume.csv")
text = text['cleaned_text'][0]

### Custom SKILL dictionary

In [2]:
# first version of the Master Skill list (the group's input)

MASTER_SKILL_LIST = [

    # Techical / Technical role
    "python", "r", "java", "javascript", "typescript",
    "c++", "c#", "scala", "go", "matlab",
    "bash", "shell scripting",
    "software engineering",
    "software development",
    "full stack development",
    "frontend development",
    "backend development",
    "api design",
    "rest apis",
    "microservices",
    "distributed systems",
    "scalable systems",
    "cloud infrastructure",
    "cloud computing",
    "cloud native",
    "cloud platforms",

    # Data Analytics
    "sql", "nosql", "postgresql", "mysql", "oracle", "sqlite",
    "mongodb", "snowflake", "redshift", "bigquery", "azure sql",
    "data analysis", "data analytics", "statistical analysis",
    "business intelligence", "operational reporting",
    "process mapping",  "requirements analysis",
    "risk management", "financial reporting",

    # Data Tools
    "pandas", "numpy", "scipy", "matplotlib", "seaborn",
    "plotly", "pyspark", "spark", "hadoop", "hive", "mapreduce", "jira"

    # Machine Learning
    "machine learning", "deep learning", "neural networks",
    "logistic regression", "linear regression", "random forest",
    "xgboost", "lightgbm", "catboost",
    "svm", "knn", "decision trees", "pca", "kmeans",
    "gradient boosting", "model tuning", "feature engineering",

    # NLP
    "nlp", "natural language processing", "topic modeling",
    "lda", "lsa", "keyword extraction",
    "named entity recognition", "text classification",
    "sentiment analysis", "embeddings", "bert", "word2vec",

    # Cloud
    "aws", "azure", "gcp", "docker", "kubernetes",
    "lambda", "ec2", "s3", "athena", "dynamodb",
    "databricks", "airflow", "cloud functions",

    # BI Tools
    "tableau", "power bi", "metabase", "looker", "qlik",
    "data visualization", "dashboard development",

    # ETL / Pipelines
    "etl", "elt", "data pipeline", "data ingestion",
    "data cleaning", "data transformation", "data integration",

    # Version Control & DevOps
    "git", "github", "gitlab", "bitbucket",
    "ci/cd", "jenkins",

    # Enterprise Tools
    "sap", "sap erp", "salesforce", "salesforce crm",
    "hubspot", "hubspot crm", "airtable", "jira", "confluence", "notion",

    # BUSINESS & ANALYTICS SKILLS
    "business analysis", "requirements gathering",
    "market research", "competitive analysis",
    "financial analysis", "risk analysis", "cost analysis",
    "forecasting", "trend analysis", "variance analysis",
    "p&l management", "strategic planning",
    "business modeling", "stakeholder management",
    "reporting", "presentation development",
    "process improvement", "process optimization",
    "root cause analysis", "gap analysis",
    "workflow automation", "operational efficiency",
    "kpi analysis", "performance analysis",
    "customer segmentation", "persona development",
    "data-driven decision making",

    # Consulting skills
    "problem solving", "insights synthesis",
    "client communication", "proposal writing",
    "project scoping", "roadmap planning",
    "change management", "cross-functional collaboration",


    # Marketing/ Sales
    "crm management", "lead generation", "pipeline management",
    "sales operations", "sales strategy", "sales forecasting",
    "revenue operations", "revops", "gtm strategy",
    "go-to-market", "account management",
    "client success", "customer retention", "digital marketing",
    "content marketing", "seo", "sem", "ppc", "email marketing",
    "campaign optimization", "social media analytics",

    # Marketing tools
    "marketing automation", "google analytics",
    "google ads", "mailchimp", "marketo",
    "outreach", "gong", "zoominfo",

    # RevOps Processes
    "validation rules", "crm integrations",
    "funnel analysis", "data stamping",

    # PRODUCT SKILLS
    "product management", "product analytics",
    "a/b testing", "experiment design",
    "feature prioritization", "user research", "ux research",
    "user stories", "agile", "scrum", "kanban",
    "roadmap development", "user journey mapping",
    "requirements documentation",
    "market sizing", "competitive positioning",

    # FINANCE & OPERATIONS SKILLS
    "fp&a", "financial modeling", "budgeting",
    "scenario analysis", "invoice processing",
    "billing operations", "revenue analysis",
    "cost optimization",

    # Operations & Supply Chain
    "supply chain management", "inventory management",
    "logistics", "procurement", "vendor management",
    "operations management", "kpi reporting",

    # SOFT SKILLS

    "communication", "leadership", "teamwork",
    "collaboration", "critical thinking", "problem solving",
    "adaptability", "time management",
    "presentation skills", "negotiation",
    "public speaking", "project management",
    "detail oriented", "strategic thinking",
    "multitasking", "analytical thinking",
    "decision making", "organization skills",
    "attention to detail", "stakeholder communication",
    "conflict resolution", "problem-solving skills",
    "relationship building","coaching", "mentoring"
]

# ---------------------------------------
# Extra skills / Tools (multi-industry)
# ---------------------------------------
EXTRA_SKILLS = [


    # Programming/ tech
    "django", "flask", "fastapi",
    "react", "react native", "angular", "vue.js", "next.js",
    "node.js", "express.js",
    "php", "ruby", "ruby on rails",
    "swift", "kotlin", "objective-c",
    "c", "perl", "rust", "haskell",

    # Mobile / app
    "android development", "ios development",
    "xcode", "android studio",

    # Testing / QA
    "unit testing", "integration testing",
    "qa testing", "automation testing",
    "selenium", "cypress", "pytest", "junit",

    # Security / networking
    "network security", "firewall configuration",
    "penetration testing", "vulnerability assessment",
    "siem", "splunk", "wireshark",
    "ssl", "tls", "vpn",

    # Data analytics
    "excel", "microsoft excel",
    "vlookup", "pivot tables",
    "google sheets",
    "sql server", "db2",

    "sas", "stata", "spss",
    "power query", "power pivot",

    "mode analytics", "lookml",
    "amplitude", "mixpanel",
    "hex", "metabase",

    # Cloud/ DevOps/ Infra
    "terraform", "ansible", "chef", "puppet",
    "github actions", "circleci", "travis ci",

    "aws lambda", "aws rds", "aws ecs", "aws ecr",
    "aws glue", "aws athena", "aws redshift",
    "azure data factory", "azure databricks",
    "gcp pubsub", "gcp dataflow", "gcp dataproc",


    # Product/ Design/ UX
    "figma", "sketch", "adobe xd",
    "invision", "balsamiq",
    "user journey mapping", "service blueprinting",
    "design thinking", "wireframing", "prototyping",
    "usability testing", "user interviews", "heuristic evaluation",


    # Marketing / Growth
    "meta ads manager", "facebook ads", "instagram ads",
    "tiktok ads", "linkedin ads",
    "google tag manager", "google search console",
    "seo keyword research", "on-page seo", "technical seo",
    "crm campaigns", "lifecycle marketing",
    "marketing funnel analysis", "conversion rate optimization",
    "ab testing", "landing page optimization",

    # Email / automation
    "klaviyo", "hubspot marketing", "salesforce marketing cloud",
    "customer.io", "braze", "iterable",

    # E-commerce
    "shopify", "woocommerce", "bigcommerce", "magento",
    "product catalog management", "pricing optimization",
    "merchandising", "inventory planning",


    # Sales/ customer success/ RevOps
    "salesforce administration", "salesforce reporting",
    "salesforce dashboards", "salesforce flows",
    "cpq", "quote to cash",
    "salesforce service cloud", "salesforce sales cloud",
    "hubspot sales", "pipedrive", "zoho crm",
    "microsoft dynamics 365",
    "outreach", "salesloft", "apollo",
    "gong", "chorus", "zoominfo",
    "cold calling", "cold emailing",
    "account planning", "territory planning",
    "renewal management", "upsell strategy",
    "churn analysis",


    # Finance/ Accounting
    "accounts payable", "accounts receivable",
    "general ledger", "reconciliation",
    "month-end close", "year-end close",
    "cash flow forecasting", "variance analysis",
    "quickbooks", "xero", "netsuite",
    "sap fico", "oracle ebs", "oracle fusion",
    "financial statement analysis",
    "credit risk modeling", "valuation modeling",
    "discounted cash flow", "dcf modeling",
    "equity research", "portfolio analysis",


    # HR/ People
    "recruiting", "candidate screening",
    "interview scheduling", "offer negotiation",
    "onboarding", "offboarding",
    "performance review process",
    "succession planning",
    "compensation analysis", "benefits administration",
    "workday", "workday hcm",
    "sap successfactors", "oracle hcm",
    "bamboohr", "greenhouse", "lever", "jobvite",

    # Operation / Logistic
    "demand planning", "capacity planning",
    "production scheduling", "quality control",
    "lean manufacturing", "six sigma", "kaizen",
    "5s methodology", "root cause analysis",
    "warehouse management", "route optimization",
    "fleet management", "last mile delivery",
    "order fulfillment", "inventory forecasting",
    "sap mm", "sap sd", "sap pp",
    "oracle scm", "manhattan wms",

    # Healthcare
    "electronic medical records", "emr systems",
    "ehr systems", "epic systems", "cerner",
    "icd-10 coding", "cpt coding",
    "clinical trials", "gcp compliance", "good clinical practice",
    "fda regulations", "hipaa compliance",
    "lab information systems", "pharmacovigilance",


    # Education
    "curriculum development", "lesson planning",
    "classroom management", "learning management systems",
    "moodle", "canvas lms", "blackboard lms",
    "online course design", "instructional design",

    # Legal

    "contract review", "contract drafting",
    "regulatory compliance", "policy development",
    "risk assessment", "internal controls",
    "gdpr compliance", "sox compliance",
    "kyd", "kyc", "aml monitoring", "anti-money laundering",


    # Creative/ Media

    "adobe photoshop", "adobe illustrator",
    "adobe indesign", "adobe premiere pro", "after effects",
    "video editing", "photo editing",
    "storyboarding", "script writing",
    "content strategy", "content calendar",
    "social media content creation",


    # Hospitality/ Retail

    "pos systems", "reservation systems",
    "inventory counting", "food safety",
    "barista skills", "cash handling",
    "customer check-in", "front desk operations",
    "event planning", "banquet operations",


    # Construction/ Engineer
    "autocad", "revit", "solidworks",
    "project bidding", "site inspection",
    "blueprint reading", "quantity surveying",
    "building codes", "osha compliance",
    "pmp", "primavera p6", "ms project",

    # Daya privacy
    "itil framework", "incident management",
    "change management process", "service desk operations",
    "access control", "identity management",


    # Language
    "translation", "interpretation",
    "bilingual communication", "multilingual support"
]




MASTER_SKILL_LIST = list(set(MASTER_SKILL_LIST + EXTRA_SKILLS))

In [3]:
print("Base master skills:", len(MASTER_SKILL_LIST))

Base master skills: 530


##  Skill matcher

In [4]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 141.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import numpy as np
import spacy
from spacy.matcher import PhraseMatcher

nlp = spacy.load("en_core_web_sm")


def build_skill_ner(skill_list):
    """
    Builds a spaCy PhraseMatcher for custom skill extraction.
    """
    matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

    # Create spaCy Doc patterns for each skill phrase
    patterns = [nlp.make_doc(skill) for skill in skill_list]
    matcher.add("SKILL", patterns)

    return matcher

skill_matcher = build_skill_ner(MASTER_SKILL_LIST)

In [6]:
def extract_skill_entities(text):
    """
    Extracts skill entities from text using the SKILL PhraseMatcher.
    Returns a unique list of skills (lowercased).
    """
    doc = nlp(text)
    matches = skill_matcher(doc)

    skills_found = set()

    for match_id, start, end in matches:
        span = doc[start:end]
        skills_found.add(span.text.lower())

    return sorted(list(skills_found))

In [7]:
skills_in_resume = extract_skill_entities(text)
print("Extracted Skills:")
print(skills_in_resume)

Extracted Skills:
['communication', 'data analysis', 'docker', 'etl', 'excel', 'forecasting', 'git', 'knn', 'lda', 'logistic regression', 'market research', 'metabase', 'mongodb', 'nosql', 'operational efficiency', 'pandas', 'postgresql', 'power query', 'python', 'r', 'sap', 'sap erp', 'sql', 'topic modeling', 'trend analysis', 'xgboost']


### Custom SKILL recognizer using PhraseMatcher

In [8]:
import spacy
from spacy import displacy
from IPython.display import HTML
spacy.displacy.render.__globals__["is_in_jupyter"] = lambda: False


class HTMLVisualizer:
    def __init__(self, html):
        self.html = html

    def _repr_html_(self):
        return self.html


def visualize_skill_entities(text):
    doc = nlp(text)
    matches = skill_matcher(doc)

    ents = []
    for match_id, start, end in matches:
        span = doc[start:end]
        ents.append({
            "start": span.start_char,
            "end": span.end_char,
            "label": "SKILL"
        })

    doc_dict = {"text": text, "ents": ents}

    html = displacy.render(doc_dict, style="ent", manual=True)

    return HTMLVisualizer(html)

visualize_skill_entities(text)

In [9]:
from spacy import displacy
from IPython.display import HTML

def visualize_all_entities(text):
    """
    Visualize BOTH:
    - spaCy NER (ORG, DATE, GPE, etc.)
    - custom SKILL entities
    """
    doc = nlp(text)

    # Default spaCy entities
    ents = [{
        "start": ent.start_char,
        "end": ent.end_char,
        "label": ent.label_
    } for ent in doc.ents]

    # Add custom SKILL entities
    matches = skill_matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        ents.append({
            "start": span.start_char,
            "end": span.end_char,
            "label": "SKILL"
        })

    # Sort entities by start index
    ents = sorted(ents, key=lambda x: x["start"])

    # Build manual entity rendering dict
    doc_dict = {"text": text, "ents": ents}

    # --- SAFE HTML rendering (spaCy will NOT import display now) ---
    html = displacy.render(doc_dict, style="ent", manual=True)

    return HTML(html)

visualize_all_entities(text)

## Read cleaned job description file

In [10]:
jobs_df = pd.read_csv("cleaned_job_data_dedup.csv")

JOB_TEXT_COL = "job_text_cleaned"

print("Shape:", jobs_df.shape)
print("Columns:", jobs_df.columns)

Shape: (14760, 2)
Columns: Index(['job_id', 'job_text_cleaned'], dtype='object')


### NER skill extractor on each job description

In [11]:
# randomly pick 100 jobs (use for testing purposes)
#jobs_df_sample = jobs_df.sample(n=100, random_state=64)

#JOB_TEXT_COL = "job_text_cleaned"

### PhraseMatcher for skills

In [12]:
# use the same extract_skill_entities function

In [13]:
texts = jobs_df[JOB_TEXT_COL].fillna("").tolist()

all_skills = []

for doc in nlp.pipe(texts, batch_size=50):
    doc_skills = []

    # use the same matcher
    matches = skill_matcher(doc)

    for match_id, start, end in matches:
        span = doc[start:end]
        doc_skills.append(span.text.strip())

    # remove duplicates for this JD
    doc_skills = list(set(doc_skills))
    all_skills.append(doc_skills)

# attach to the sample dataframe
jobs_df["skills_extracted"] = all_skills
jobs_df["n_skills"] = jobs_df["skills_extracted"].apply(len)

In [14]:
jobs_df.head(5)

,job_id,job_text_cleaned,skills_extracted,n_skills
0,0,Digital Marketing Specialist\nManage and grow ...,"[Digital Marketing, Social media analytics]",2
1,1,Web Developer\nDesign and code user interfaces...,"[Angular, React, JavaScript]",3
2,2,Operations Manager\nEstablish and enforce qual...,"[Root cause analysis, Quality Control, Quality...",4
3,3,"Network Engineer\nDesign, configure, and optim...",[],0
4,4,Event Manager\nSpecialize in conference and co...,"[logistics, Event planning, budgeting]",3


In [15]:
jobs_df['n_skills'].min()

0

In [16]:
jobs_df['n_skills'].max()

52

#### SKILL-only visualization

In [17]:
import spacy
from spacy import displacy
from IPython.display import HTML

# Make displacy work nicely inside Jupyter
spacy.displacy.render.__globals__["is_in_jupyter"] = lambda: True

class HTMLVisualizer:
    def __init__(self, html):
        self.html = html

    def _repr_html_(self):
        return self.html

def visualize_skill_entities(text):
    """
    Visualize the skills detected by skill_matcher
    inside a block of job description text.
    """

    doc = nlp(text)
    matches = skill_matcher(doc)

    ents = []
    for match_id, start, end in matches:
        span = doc[start:end]
        ents.append({
            "start": span.start_char,
            "end": span.end_char,
            "label": "SKILL"
        })

    # displaCy manual mode
    doc_dict = {
        "text": text,
        "ents": ents,
        "title": None
    }

    html = displacy.render(doc_dict, style="ent", manual=True)

    return HTMLVisualizer(html)

In [18]:
visualize_skill_entities(jobs_df["job_text_cleaned"].iloc[4])

#### All Entities visualization

In [19]:
# just in case

In [20]:
import spacy
from spacy import displacy
from IPython.display import HTML

spacy.displacy.render.__globals__["is_in_jupyter"] = lambda: True

def visualize_job_entities(text):
    """
    Visualize BOTH:
    - spaCy NER entities (ORG, DATE, GPE, PERSON, MONEY, PRODUCT, NORP, etc.)
    - Custom SKILL entities from skill_matcher
    """

    doc = nlp(text)

    # ----- 1) spaCy default NER entities -----
    ents = [{
        "start": ent.start_char,
        "end": ent.end_char,
        "label": ent.label_
    } for ent in doc.ents]

    # ----- 2) Add custom SKILL entities -----
    matches = skill_matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        ents.append({
            "start": span.start_char,
            "end": span.end_char,
            "label": "SKILL"
        })

    # Sort entities by start index for clean display
    ents = sorted(ents, key=lambda x: x["start"])

    # ----- 3) Build displaCy structure -----
    doc_dict = {
        "text": text,
        "ents": ents,
        "title": None
    }

    html = displacy.render(doc_dict, style="ent", manual=True)
    return HTML(html)

In [21]:
visualize_job_entities(jobs_df["job_text_cleaned"].iloc[3])

<IPython.core.display.HTML object>

# SkillCore with Jaccard Similarity

In [22]:
def skill_jaccard_score(resume_skills, job_skills):
    """
    Jaccard similarity between resume skills and job skills.
    = overlap / union
    """

    resume_set = set(resume_skills)
    job_set = set(job_skills)

    # If both are empty, return 0
    union = resume_set | job_set
    if not union:
        return 0.0

    overlap = resume_set & job_set

    score = len(overlap) / len(union)
    return score


In [23]:
jobs_df["SkillScore"] = jobs_df["skills_extracted"].apply(
    lambda job_sk: skill_jaccard_score(skills_in_resume, job_sk)
)

# Add resume skills column (same for every row)
jobs_df["resume_skills"] = [skills_in_resume] * len(jobs_df)


top_matches = (
    jobs_df
    .sort_values("SkillScore", ascending=False)
    [[JOB_TEXT_COL, "skills_extracted", "resume_skills", "SkillScore"]]
    .head(10)
)

In [24]:
# pd.set_option('display.max_colwidth', None)

In [25]:
top_matches

job_text_cleaned  \
9830                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

In [26]:
bottom_matches = (
    jobs_df
    .sort_values("SkillScore", ascending=True)
    [[JOB_TEXT_COL, "skills_extracted", "resume_skills", "SkillScore"]]
    .head(10)
)

In [27]:
bottom_matches

job_text_cleaned  \
6365   Fullstack Tech lead - Trust & Safety (All Genders) with verification\nAbout the job Dailymotion c'est plus qu’une app vidéo, c’est une conversation visuelle en mouvement qui repose sur un algorithme unique pensé pour développer l'esprit critique des utilisateurs. Dailymotion apporte de la nuance aux débats qui animent la jeunesse et remet l’écoute, la découverte et la bienveillance au cœur des échanges pour contribuer à construire un Internet plus sûr et vertueux. Dailymotion, c'est aussi "Dailymotion Pro”, une solution d'hébergement et de diffusion vidéo au service des professionnels de tous les secteurs ; et "Dailymotion Advertising", une plateforme vidéo marketing propriétaire et puissante, qui propose un environnement qualitatif et sûr pour les marques. Notre équipe se compose de 400 collaborateurs en France, à New York et à Singapore, tous unis par la même ambition : faire bouger visiblement les lignes de l’écosystème des plateformes vidéo dans le monde. Ce qui nous lie? A Dailymotion, notre équipe composée de talents de plus de 40 nationalités évolue chaque jour en portant quatre valeurs fondamentales: Ouvrir des perspectives - Nous créons des ponts entre des humains de tous horizons. Nous croyons à la liberté d’expression, à l'inclusion et au mélange des cultures. Bâtir un monde plus sûr - Nous cultivons l'empathie, clé pour comprendre l'altérité. Prendre soin les uns des autres est notre moteur pour créer une communauté solidaire et sûre. Faire preuve d'audace - Nous sommes des challengers ambitieux. Nous faisons des choix à contre-courant qui auront un impact visible dans l’écosystème des plateformes. Créer du sens - Nous suscitons la joie et la curiosité, nous nous sentons bien dans ce que nous faisons et nous voulons que tous les autres ressentent la même satisfaction. Job Description Contexte : Dans le cadre du renforcement de notre pôle Trust & Safety, nous recherchons un Tech Lead fullstack pour encadrer le développement des outils techniques dédiés à la modération et la conformité des contenus sur notre plateforme. Vous jouerez un rôle clé dans la conception, l'amélioration et l'industrialisation de nos systèmes internes (détection, workflow, reporting...), en collaboration étroite avec l’équipe Trust & Safety Operations, les équipes produit, data et juridique. Missions : Piloter la roadmap technique des outils Trust & Safety : détection de contenus, automatisation des workflows, modération, tableaux de bord, alerting. Designer et construire des systèmes backend en utilisant une architecture micro services Design and build API-centric systems using REST, GraphQL, and OpenAPI standards Design and build queueing and event-driven systems Participate in legacy system modernization and incremental migration strategies Proactivity on evolutions and improvements Concevoir des solutions pragmatiques à fort impact, en optimisant les ressources et en priorisant selon la valeur métier. Exploiter l’IA pour : Améliorer la détection automatisée de contenus problématiques (violence, haine, spam...). Fluidifier les processus internes via l’automatisation (workflow, tagging, priorisation...). Accélérer le développement (utilisation d’outils comme Cursor AI, génération de code assistée...). Garantir la qualité, la sécurité et la maintenabilité des outils déployés. Encadrer une équipe de 3 développeurs front-end et back-end Profil recherché : Expérience significative en tant que Tech Lead, idéalement dans un environnement tech à fort trafic (social media, plateforme vidéo...). Solide culture back-end, avec une bonne compréhension des enjeux front et d’UX interne. À l’aise avec l’utilisation d’outils d’IA générative dans le cycle de développement (détection, automatisation, assistance au code...). Esprit pragmatique, orienté impact et solutions, capable d’arbitrer entre rapidité, efficacité et scalabilité. App

In [28]:
jobs_df_final = jobs_df[["job_text_cleaned", "SkillScore"]]

# Final df before normalize the score

In [30]:
jobs_df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14760 entries, 0 to 14759
Data columns (total 2 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   job_text_cleaned  14760 non-null  object 
 1   SkillScore        14760 non-null  float64
dtypes: float64(1), object(1)
memory usage: 230.8+ KB


## Normalize the SkillScore

In [32]:
# min and max SkillScore
min_s = jobs_df_final["SkillScore"].min()
max_s = jobs_df_final["SkillScore"].max()

# Avoid division by zero
if max_s == min_s:
    jobs_df_final["SkillScore_normalized"] = 0.0
else:
    jobs_df_final["SkillScore_normalized"] = (
        (jobs_df_final["SkillScore"] - min_s) / (max_s - min_s)
    )

/tmp/ipython-input-2533698986.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jobs_df_final["SkillScore_normalized"] = (


In [33]:
# New normalized dataframe
jobs_df_normalized = jobs_df_final[[
    "job_text_cleaned",
    "SkillScore_normalized"
]].copy()

jobs_df_normalized.head()

,job_text_cleaned,SkillScore_normalized
0,"Digital Marketing Specialist\nManage and grow social media accounts, create engaging content, and interact with the online community. Develop social media content calendars and strategies. Monitor social media trends and engagement metrics.\nSocial Media Managers oversee an organizations social media presence. They create and schedule content, engage with followers, and analyze social media metrics to drive brand awareness and engagement.\nSocial media platforms (e.g., Facebook, Twitter, Instagram) Content creation and scheduling Social media analytics and insights Community engagement Paid social advertising",0.0
1,"Web Developer\nDesign and code user interfaces for websites, ensuring a seamless and visually appealing user experience. Collaborate with UX designers to optimize user journeys. Ensure cross-browser compatibility and responsive design.\nFrontend Web Developers design and implement user interfaces for websites, ensuring they are visually appealing and user-friendly. They collaborate with designers and backend developers to create seamless web experiences for users.\nHTML, CSS, JavaScript Frontend frameworks (e.g., React, Angular) User experience (UX)",0.0
2,"Operations Manager\nEstablish and enforce quality control standards and procedures. Conduct quality audits and inspections. Collaborate with production teams to address quality issues and implement improvements.\nQuality Control Managers establish and enforce quality standards within an organization. They develop quality control processes, perform inspections, and implement corrective actions to maintain product or service quality.\nQuality control processes and methodologies Statistical process control (SPC) Root cause analysis and corrective action Quality management systems (e.g., ISO 9001) Compliance and regulatory knowledge",0.0
3,"Network Engineer\nDesign, configure, and optimize wireless networks, ensuring reliable and secure wireless connectivity. Troubleshoot wireless network issues. Plan and implement wireless network upgrades.\nWireless Network Engineers design, implement, and maintain wireless network solutions. They optimize wireless connectivity, troubleshoot issues, and ensure reliable and secure wireless communications.\nWireless network design and architecture Wi-Fi standards and protocols RF (Radio Frequency) planning and optimization Wireless security protocols Troubleshooting wireless network issues",0.0
4,"Event Manager\nSpecialize in conference and convention planning. Coordinate speaker sessions, exhibitors, and attendee experiences. Oversee event registration and marketing.\nA Conference Manager coordinates and manages conferences, meetings, and events. They plan logistics, handle budgeting, liaise with vendors, and ensure the smooth execution of events, catering to the needs and expectations of attendees.\nEvent planning Conference logistics Budget management Vendor coordination Marketing and promotion Client relations",0.0


In [35]:
top10_normalized = (
    jobs_df_normalized
    .sort_values("SkillScore_normalized", ascending=False)
    .head(10)
)

top10_normalized

job_text_cleaned  \
9830                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        

# Final Normalized SkillScore df

In [38]:
jobs_df_normalized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14760 entries, 0 to 14759
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   job_text_cleaned       14760 non-null  object 
 1   SkillScore_normalized  14760 non-null  float64
dtypes: float64(1), object(1)
memory usage: 230.8+ KB
